# Basic Use Case: Summarize a Text File

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

groq_api_key = os.environ["GROQ_API_KEY"]

# Setup LLM to use, gemma using Groq

In [15]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="gemma-7b-it",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## Load The text file

In [5]:
with open('data/be-good-and-how-not-to-die.txt', 'r') as file:
    article = file.read()


print(f'Type of file content: {type(article)}')
print(article[:285])


Type of file content: <class 'str'>
Be good

April 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the
phrase that became our motto: Make something people want.  We've
learned a lot since then, but if I were choosing now that's still
the one 


In [16]:
num_tokens = llm.get_num_tokens(article)
print(num_tokens)

6391


##  Gemma 7B model is typically 4096 tokens
We cannot process the entire article in one chunk, we need to split it.

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [19]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], 
    chunk_size=5000,
    chunk_overlap=350
)
article_chunks = text_splitter.create_documents([article])
print(f"You have {len(article_chunks)} chunks instead of 1 article.")

You have 8 chunks instead of 1 article.


## Three ways to summarize or otherwise combine documents.

- Stuff, which simply concatenates documents into a prompt;
- Map-reduce, which splits documents into batches, summarizes those, and then summarizes the summaries;
- Refine, which updates a rolling summary be iterating over the documents in a sequence.

In [43]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce"
)

In [44]:
article_summary = chain.invoke(article_chunks)


In [45]:
print(article_summary["output_text"])

**Summary:**

The essay explores the tension faced by startups between generating financial gain and creating impactful products. It argues that prioritizing user value and creating something meaningful is more valuable than immediate profit. The author emphasizes the importance of following a "do-good" philosophy, which fosters morale, attracts support, and promotes success. The essay highlights the potential for startups to address societal challenges and emphasizes the importance of transparency and accountability in business practices. The author encourages perseverance and resilience in the face of challenges, emphasizing that setbacks are part of the startup journey.
